## Apiux & SII: Transformacion de contribuyentes con alertas tributarias a contaminados
## ATENCION: proyecto sujeto a mantenimiento continuo. 

## Henry Vega (henrry.vega@api-ux.com)
## Data analyst

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
from pyspark.sql.types import IntegerType,BooleanType,DateType,StringType

In [2]:
spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.kerberos.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/") \
  .config("spark.executor.memory", "16g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "2") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()
warnings.filterwarnings('ignore', category=DeprecationWarning)
sc=spark.sparkContext
sc.setLogLevel ('ERROR')
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")

Setting spark.hadoop.yarn.resourcemanager.principal to yasser.nanjari


In [4]:
Carpeta = "Proceso Renta 2024/"
Archivo = "ALERTAS_ACTIVAS_2023-2.csv"
Carpeta_Salida = "Renta 2024/"
PATH = "../../"
ALERTAS = "4110, 4111, 4112, 4113"

csv_path = PATH+"data/raw/"+Carpeta+Archivo
a = spark.read.option("header", "true").option("delimiter", ";").csv(csv_path)

In [5]:
#Columnas del dataframe
a.printSchema()

root
 |-- CONT_RUT_E: string (nullable = true)
 |-- ALER_COD_TIPO_ALERTA: string (nullable = true)



In [6]:
a.createOrReplaceTempView("alertas")

In [7]:
spark.sql(f'SELECT * FROM alertas WHERE ALER_COD_TIPO_ALERTA IN ({ALERTAS})').createOrReplaceTempView("alertas")

Por lo cual, tenemos 25647 contribuyentes contaminados con alguna alerta. Se exportara el dataset con un valor 1 como score para estos contribuyentes. 

In [8]:
out=spark.sql("SELECT CONT_RUT_E as CONT_RUT, 1 as score from alertas").toPandas()
out.reset_index(drop=True).to_csv(f'/home/cdsw/data/processed/{Carpeta_Salida}contaminados.csv',index=False)  